# This note book shows how to load header file and waveform data and plot it.

In [1]:
import pandas as pd
from datetime import datetime
import os
import pickle
from datetime import timedelta
from datetime import datetime
from datetime import date
from datetime import time
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import os

import json

import sys
from scipy.io import loadmat

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
import matplotlib.ticker as plticker

import seaborn as sns


import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PhysioMC/') # add this line so Data and data are visible in this file

# from PatchWand import *
from filters import *
from setting import *
# from preprocessing import *
from ECG_module import *
from dataIO import *
from stage1_PPG_analysis import *

from importlib import reload
%load_ext autoreload
%autoreload 2

In [2]:
# %pip install feather-format

In [3]:
# %pip install parfor

In [4]:
selected_id = 5

In [ ]:
%pwd

# load csv files David prepared

## load the demographic data, only look at black or white subjects for now

In [6]:
demographic_filedir = '/home/mchan82/kamaleswaranlab/Sepsis3_062321/dlin_analysis/Pat_BedTime_List_some_on_BMI_2_6_22_dem.csv'
df_demographic = pd.read_csv(demographic_filedir, index_col=0)  

# mask out subjects who are not black and white
mask = (df_demographic['Race String']=='African American  or Black') | (df_demographic['Race String']=='Caucasian or White')

print('only {:.2f}% is black or white'.format(mask.mean()*100))
df_demographic = df_demographic[mask]

only 43.47% is black or white


In [7]:
df_demographic

,deBedTime,DateStart,DateEnd,PAT_ID,BED_LOCATION_START,BED_LOCATION_END,White,Black,Race String
0,A118-0520880083,2018-03-12 18:41:23,2018-03-12 23:13:00,18004189.0,2018-03-12 18:41:39,03/14/2018 20:20:00,False,True,African American or Black
1,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,01/18/2018 21:15:32,True,False,Caucasian or White
3,A088-0516735120,2018-01-23 19:18:40,2018-01-23 19:43:15,17395882.0,2018-01-23 19:17:04,01/26/2018 20:42:23,True,False,Caucasian or White
5,A051-0495488308,2017-05-22 21:25:08,2017-06-02 15:53:25,17655916.0,2017-05-22 21:21:50,06/02/2017 12:51:00,True,False,Caucasian or White
8,A121-0520783579,2018-03-11 15:52:58,2018-03-11 15:53:14,91393403.0,2018-03-11 15:48:46,03/14/2018 18:44:32,True,False,Caucasian or White
...,...,...,...,...,...,...,...,...,...
6266,A134-0515603037,2018-01-10 16:50:37,2018-01-31 04:03:15,49764871.0,2018-01-31 01:58:06,01/31/2018 21:23:39,False,True,African American or Black
6268,A119-0515603035,2018-01-10 16:50:35,2018-02-06 19:00:52,92559329.0,2018-02-06 21:53:05,02/07/2018 23:51:17,False,True,African American or Black
6269,A113-0515603017,2018-01-10 16:50:16,2018-03-03 01:42:37,96901673.0,2017-12-12 15:59:54,03/02/2018 17:03:03,False,True,African American or Black
6270,A020-0509637604,2017-11-02 15:46:44,2017-12-05 20:08:36,98391127.0,2017-12-05 18:12:04,12/11/2017 18:43:14,False,True,African American or Black


## load the oxygen saturation data
### TODO: need to add FiO2 and PaO2 data

In [8]:
# FiO2 and PaO2? (PaO2 / FiO2 < 400 indicates respiratory problem)
oxygen_sat_filedir = '/home/mchan82/kamaleswaranlab/Sepsis3_062321/dlin_analysis/Pat_BedTime_List_some_on_BMI_2_6_22_spo2_sao2.csv'
df_oxygen_sat = pd.read_csv(oxygen_sat_filedir, index_col=0)  

In [9]:
mask = (df_oxygen_sat['Black']==1) | (df_oxygen_sat['White']==1)
df_oxygen_sat = df_oxygen_sat[mask]

In [10]:
df_oxygen_sat['times'] =  pd.to_datetime(df_oxygen_sat['times'], format="%Y-%m-%d %H:%M:%S")


In [11]:
df_oxygen_sat = df_oxygen_sat.rename(columns = {'pat_id':'PAT_ID', 'csn':'CSN'})
df_oxygen_sat

,index,spo2,sao2,times,Black,White,PAT_ID,CSN,csn_idx
0,4,97.0,96.5,2017-07-17 12:29:03,0.0,1.0,1842529.0,3885207198,45128.0
1,5,98.0,96.5,2017-07-17 13:29:03,0.0,1.0,1842529.0,3885207198,45128.0
2,6,95.0,96.5,2017-07-17 14:29:03,0.0,1.0,1842529.0,3885207198,45128.0
3,7,97.0,96.5,2017-07-17 15:29:03,0.0,1.0,1842529.0,3885207198,45128.0
4,8,97.0,96.5,2017-07-17 16:29:03,0.0,1.0,1842529.0,3885207198,45128.0
...,...,...,...,...,...,...,...,...,...
27,30,100.0,98.9,2018-03-29 08:00:47,1.0,0.0,99962669.0,58104308087,107482.0
28,31,100.0,98.9,2018-03-29 09:00:47,1.0,0.0,99962669.0,58104308087,107482.0
29,32,100.0,98.9,2018-03-29 10:00:47,1.0,0.0,99962669.0,58104308087,107482.0
30,33,98.0,98.9,2018-03-29 11:00:47,1.0,0.0,99962669.0,58104308087,107482.0


In [12]:
df_oxygen_sat.describe()

,index,spo2,sao2,Black,White,PAT_ID,CSN,csn_idx
count,200960.000000,200960.000000,200960.000000,200960.000000,200960.000000,2.009600e+05,2.009600e+05,200960.000000
mean,269.355608,97.045106,96.870626,0.408360,0.591640,6.553404e+07,3.652402e+10,48579.365013
std,361.627686,4.305423,3.633344,0.491532,0.491532,3.403954e+07,2.047750e+10,29484.596870
min,0.000000,0.000000,26.550000,0.000000,0.000000,1.842529e+06,6.669707e+07,73.000000
25%,45.000000,96.000000,96.200000,0.000000,0.000000,2.485274e+07,1.352412e+10,22993.000000
50%,136.000000,98.000000,97.900000,0.000000,1.000000,8.832823e+07,4.666620e+10,46759.000000
75%,350.000000,100.000000,98.800000,1.000000,1.000000,9.786810e+07,5.546849e+10,70885.000000
max,4426.000000,100.000000,100.750000,1.000000,1.000000,9.996267e+07,5.811763e+10,142522.000000


In [13]:
df_oxygen_sat['CSN'].unique().shape

(2290,)

## load the ind csv
### compute BED Time Diff (to show that we have much more bed data)
### show the median the time diffs

In [14]:
ind_filedir = '/home/mchan82/kamaleswaranlab/Sepsis3_062321/dlin_analysis/Pat_BedTime_List_some_on_BMI_2_6_22_ind.csv'
df_ind = pd.read_csv(ind_filedir, index_col=0)  

mask = (df_ind['Race String']=='African American  or Black') | (df_ind['Race String']=='Caucasian or White')
df_ind = df_ind[mask]

In [15]:
df_ind['DateStart'] = pd.to_datetime(df_ind['DateStart'], format="%Y-%m-%d %H:%M:%S")
df_ind['DateEnd'] = pd.to_datetime(df_ind['DateEnd'], format="%Y-%m-%d %H:%M:%S")

## TODO: change BED Time Difference

In [16]:
BED_LOCATION_START = pd.to_datetime(df_ind['BED_LOCATION_START'], format="%Y-%m-%d %H:%M:%S")
BED_LOCATION_END = pd.to_datetime(df_ind['BED_LOCATION_END'], format="%m/%d/%Y %H:%M:%S")

df_ind['BED_LOCATION_START'] = BED_LOCATION_START
df_ind['BED_LOCATION_END'] = BED_LOCATION_END


In [17]:
DateStart = pd.to_datetime(df_ind['DateStart'], format="%Y-%m-%d %H:%M:%S")
DateEnd = pd.to_datetime(df_ind['DateEnd'], format="%m/%d/%Y %H:%M:%S")

df_ind['DateStart'] = DateStart
df_ind['DateEnd'] = DateEnd

df_ind['BED Time Difference'] = (DateEnd-DateStart).dt.total_seconds()

In [18]:
print('EHR time diff: {:.2f}hr'.format(df_ind['Time Difference'].median()/60/60))
print('Waveform time diff: {:.2f}hr'.format(df_ind['BED Time Difference'].median()/60/60))

EHR time diff: -0.06hr
Waveform time diff: 42.28hr


In [19]:
df_ind

,deBedTime,DateStart,DateEnd,PAT_ID,BED_LOCATION_START,BED_LOCATION_END,White,Black,Race String,Time Difference,Time Found,True Time Found,CSN Exist Flag,CSN,BED Time Difference
0,A118-0520880083,2018-03-12 18:41:23,2018-03-12 23:13:00,18004189.0,2018-03-12 18:41:39,2018-03-14 20:20:00,False,True,African American or Black,1235.0,2018-03-12 18:20:48,2018-03-12 11:20:48,True,3.967156e+10,1.629700e+04
1,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7.998000e+03
2,A065-0489270594,2017-03-11 22:16:34,2017-03-14 14:50:58,96266988.0,2017-03-11 22:15:37,2017-03-14 10:58:53,False,True,African American or Black,1450.0,2017-03-11 21:52:24,2017-03-11 16:52:24,True,5.290317e+10,2.324640e+05
3,A088-0516735120,2018-01-23 19:18:40,2018-01-23 19:43:15,17395882.0,2018-01-23 19:17:04,2018-01-26 20:42:23,True,False,Caucasian or White,-20.0,2018-01-23 19:19:00,2018-01-22 22:19:00,True,5.754876e+10,1.475000e+03
5,A051-0495488308,2017-05-22 21:25:08,2017-06-02 15:53:25,17655916.0,2017-05-22 21:21:50,2017-06-02 12:51:00,True,False,Caucasian or White,-652.0,2017-05-22 21:36:00,2017-05-22 18:36:00,True,3.005482e+10,9.304970e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,A050-0519142630,2018-02-20 16:03:50,2018-02-20 16:10:55,92165161.0,2017-10-27 21:00:00,2017-10-28 11:54:32,True,False,Caucasian or White,9436187.0,2017-11-03 11:54:03,2017-10-27 16:54:03,True,4.521433e+10,4.250000e+02
6279,A050-0519143068,2018-02-20 16:11:08,1969-12-31 23:59:58,92165161.0,2017-10-27 21:00:00,2017-10-28 11:54:32,True,False,Caucasian or White,9436625.0,2017-11-03 11:54:03,2017-10-27 16:54:03,True,4.521433e+10,-1.519143e+09
6280,A075-0519140345,2018-02-20 15:25:45,2018-02-20 15:32:34,50293730.0,2017-10-27 15:19:55,2017-10-27 21:08:22,True,False,Caucasian or White,8883352.0,2017-11-09 19:49:53,2017-10-24 10:49:53,True,1.426466e+10,4.090000e+02
6283,A047-0519141716,2018-02-20 15:48:36,2018-02-20 15:52:32,23329164.0,2017-10-25 10:08:13,2017-10-28 11:24:36,False,True,African American or Black,9913831.0,2017-10-28 22:58:05,2017-10-18 18:58:05,True,1.089383e+10,2.360000e+02


In [20]:
df_ind['CSN'].unique().shape

(3644,)

In [21]:
df_ind['Time Difference']

0          1235.0
1         -1531.0
2          1450.0
3           -20.0
5          -652.0
          ...    
6278    9436187.0
6279    9436625.0
6280    8883352.0
6283    9913831.0
6284   -3218438.0
Name: Time Difference, Length: 5696, dtype: float64

# merge df_ind with df_oxygen_sat so I can get deBedTime, DateStart, DateEnd, PAT_ID, White	Black, CSN, SaO2, SpO2, oxy sat. times in one column

## TODO: merge using the CSN instead 

In [22]:
df_merged = df_ind.merge(df_oxygen_sat[['spo2', 'sao2', 'times', 'PAT_ID', 'CSN']], on=['CSN', 'PAT_ID'], how='outer')
df_merged

,deBedTime,DateStart,DateEnd,PAT_ID,BED_LOCATION_START,BED_LOCATION_END,White,Black,Race String,Time Difference,Time Found,True Time Found,CSN Exist Flag,CSN,BED Time Difference,spo2,sao2,times
0,A118-0520880083,2018-03-12 18:41:23,2018-03-12 23:13:00,18004189.0,2018-03-12 18:41:39,2018-03-14 20:20:00,False,True,African American or Black,1235.0,2018-03-12 18:20:48,2018-03-12 11:20:48,True,3.967156e+10,16297.0,NaN,NaN,NaT
1,A118-0520896395,2018-03-12 23:13:15,2018-03-15 03:19:11,18004189.0,2018-03-12 18:41:39,2018-03-14 20:20:00,False,True,African American or Black,-453.0,2018-03-12 23:20:48,2018-03-12 11:20:48,True,3.967156e+10,187556.0,NaN,NaN,NaT
2,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7998.0,94.500000,92.4,2018-01-10 17:16:00
3,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7998.0,96.000000,92.4,2018-01-10 18:16:00
4,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7998.0,95.333333,92.4,2018-01-10 19:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390021,A106-0515603020,2018-01-10 16:50:19,2018-03-09 03:01:20,26319659.0,2017-12-28 02:51:36,2018-03-07 13:20:00,False,True,African American or Black,1741485.0,2017-12-21 13:05:34,2017-11-08 11:05:34,True,1.129628e+10,4961461.0,99.000000,97.3,2017-12-03 01:05:34
390022,A106-0515603020,2018-01-10 16:50:19,2018-03-09 03:01:20,26319659.0,2017-12-28 02:51:36,2018-03-07 13:20:00,False,True,African American or Black,1741485.0,2017-12-21 13:05:34,2017-11-08 11:05:34,True,1.129628e+10,4961461.0,99.000000,97.3,2017-12-03 02:05:34
390023,A106-0515603020,2018-01-10 16:50:19,2018-03-09 03:01:20,26319659.0,2017-12-28 02:51:36,2018-03-07 13:20:00,False,True,African American or Black,1741485.0,2017-12-21 13:05:34,2017-11-08 11:05:34,True,1.129628e+10,4961461.0,98.000000,97.3,2017-12-03 03:05:34
390024,A112-0515603023,2018-01-10 16:50:23,2018-03-02 16:11:07,21341087.0,2017-11-30 15:02:07,2018-03-02 08:48:00,True,False,Caucasian or White,-1636.0,2018-01-10 17:17:39,2017-11-25 14:17:39,True,1.042668e+10,4404044.0,NaN,NaN,NaT


In [23]:
(df_merged['DateEnd']=='1969-12-31 23:59:58').mean()

0.030838969709711662

In [24]:
(df_merged['DateEnd']>'1970-12-31 23:59:58').mean()

0.9691610302902883

In [25]:
# df_merged[(df_merged['PAT_ID']==PAT_ID) & (df_merged['deBedTime']=='A140-0515603029')]

# But simply merging them using PAT_ID is not ideal: same PAT_ID may have several deBedTime
## need to filter out rows incorrectly align by checking the timing
## only associate the demographic data to the ox sat. data when the timing match

In [26]:
DateStart = pd.to_datetime(df_merged['DateStart'], format="%Y-%m-%d %H:%M:%S") # str to datetime
DateEnd = pd.to_datetime(df_merged['DateEnd'], format="%Y-%m-%d %H:%M:%S")
times = pd.to_datetime(df_merged['times'], format="%Y-%m-%d %H:%M:%S")

In [27]:
# timedelta: int to datetime
mask = (times-timedelta(minutes=min_threshold) >= DateStart) & (times+timedelta(minutes=min_threshold) <= DateEnd)
df_merged = df_merged[mask]

In [28]:
print(mask.mean())

0.3821847774250947


In [29]:
df_merged['CSN'].unique().shape

(2117,)

In [30]:
df_merged['PAT_ID'].unique().shape

(2011,)

In [31]:
df_merged = df_merged.reset_index(drop=True)
df_merged

,deBedTime,DateStart,DateEnd,PAT_ID,BED_LOCATION_START,BED_LOCATION_END,White,Black,Race String,Time Difference,Time Found,True Time Found,CSN Exist Flag,CSN,BED Time Difference,spo2,sao2,times
0,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7998.0,94.5,92.4,2018-01-10 17:16:00
1,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7998.0,96.0,92.4,2018-01-10 18:16:00
2,A140-0515620983,2018-01-10 21:49:43,2018-01-21 21:51:58,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1577.0,2018-01-10 22:16:00,2017-12-18 21:16:00,True,4.991014e+10,950535.0,95.0,93.9,2018-01-10 22:16:00
3,A140-0515620983,2018-01-10 21:49:43,2018-01-21 21:51:58,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1577.0,2018-01-10 22:16:00,2017-12-18 21:16:00,True,4.991014e+10,950535.0,94.0,93.9,2018-01-10 23:16:00
4,A140-0515620983,2018-01-10 21:49:43,2018-01-21 21:51:58,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1577.0,2018-01-10 22:16:00,2017-12-18 21:16:00,True,4.991014e+10,950535.0,93.5,93.9,2018-01-11 00:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149057,A063-0493131179,2017-04-25 14:39:39,2017-05-18 20:01:09,47565503.0,2017-04-12 17:38:13,2017-05-18 16:05:00,True,False,Caucasian or White,-1521.0,2017-04-25 15:05:00,2017-04-08 21:05:00,True,1.326399e+10,2006490.0,98.0,99.1,2017-05-16 01:05:00
149058,A063-0493131179,2017-04-25 14:39:39,2017-05-18 20:01:09,47565503.0,2017-04-12 17:38:13,2017-05-18 16:05:00,True,False,Caucasian or White,-1521.0,2017-04-25 15:05:00,2017-04-08 21:05:00,True,1.326399e+10,2006490.0,98.0,99.1,2017-05-16 02:05:00
149059,A063-0493131179,2017-04-25 14:39:39,2017-05-18 20:01:09,47565503.0,2017-04-12 17:38:13,2017-05-18 16:05:00,True,False,Caucasian or White,-1521.0,2017-04-25 15:05:00,2017-04-08 21:05:00,True,1.326399e+10,2006490.0,100.0,99.1,2017-05-16 03:05:00
149060,A063-0493131179,2017-04-25 14:39:39,2017-05-18 20:01:09,47565503.0,2017-04-12 17:38:13,2017-05-18 16:05:00,True,False,Caucasian or White,-1521.0,2017-04-25 15:05:00,2017-04-08 21:05:00,True,1.326399e+10,2006490.0,94.0,99.1,2017-05-16 04:05:00


In [32]:
# (24*60*60)/149062

# get PPG data

## os.listdir(inputdir) = 
## ['A071-0512777549',
## 'A063-0482186017',
## 'A113-0520303111', ...]

In [33]:
# waveform_window

## os.listdir(inputdir_rec) = 
## ['A058-0502749430_0028d.mat',
##  'A058-0502749430.almth',
##  'A058-0502749430_0013.mat',
##  'A058-0502749430_0013.hea',...]

In [34]:
outputdir = '../../data/stage1/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

In [35]:
with open('recording_lookup.json') as json_file:
    recording_lookup = json.load(json_file)


In [36]:
homedir = os.path.expanduser('~')+'/'

# this is where all waveform data are stored
inputdir = '/labs/kamaleswaranlab/Sepsis3_062321/Files_cp_sftp_6_30_21/Files_cp_6_30_21/'

# yes
# rec_id = 'A058-0502749430'

# no
# rec_id = 'A030-0497283634'
# rec_id = 'A097-0427935662'
# rec_id = 'A085-0500344504'


# # check the header files in here
# for rec_file in os.listdir(inputdir+rec_id):
#     if 'hea' in rec_file:
#         print(rec_file)

In [37]:
# TODO: move to dataIO.py, modularize it
def get_header_valid_waveform(inputdir_rec, row):

    # header_list = []
    for header_name in sorted(os.listdir(inputdir_rec)):
        
        if '.hea' not in header_name:
            continue
        if '_' not in header_name:
            continue
        if '_0n' in header_name:
            continue
        if '_1n' in header_name:
            continue
        if 'layout' in header_name:
            continue

        # print(header_name)
        headerdir = inputdir_rec + header_name


        header_id = header_name.split('_')[-1].split('.hea')[0]
        # print(header_id)
        # print(int(header_id))
        # sys.exit()

        with open(headerdir, 'r') as f:
            header = f.readlines()

        header_dict = header_extract(header)

        # print(header_name)

        allheader_hours = int(header_id)*8
        currentheader_hours = header_dict['t_dur(hr)']

        # waveform_start = row['BED_LOCATION_START'] + timedelta(hours=allheader_hours)
        # waveform_end = row['BED_LOCATION_START'] + timedelta(hours=allheader_hours) + timedelta(hours=currentheader_hours)

        # using DateStart since it is the start time of the waveform
        waveform_start = row['DateStart'] + timedelta(hours=allheader_hours)
        waveform_end = row['DateStart'] + timedelta(hours=allheader_hours) + timedelta(hours=currentheader_hours)

        header_dict['waveform_start'] = waveform_start
        header_dict['waveform_end'] = waveform_end
        header_dict['header_name'] = header_name
        header_dict['header_id'] = header_id

        # header_list.append(header_dict)
        
    # return header_list
        if (waveform_start > row['times']) or (waveform_end < row['times']):
            # print('wrong recording')
            continue
        else:
            # print(header_start, header_end, row['times'])
            # print('bingo!')
            return header_dict


        return None
        # print('\t', header_dict['start_time'], header_dict['date'])

    # header_id = header_name.split('_')[-1]

In [38]:
outputdir_recording = outputdir+'waveform/'
if not os.path.exists(outputdir_recording):
    os.makedirs(outputdir_recording)
    


In [39]:
# df_demographic_processed = pd.read_csv(outputdir_recording+'df_demographic_processed.csv.gz').sort_values('index').reset_index(drop=True)


In [40]:
# df_demographic_processed.shape[0]/df_merged.shape[0]

# TODO
## 1. filter the signal first (DONE)
## 2. apply QRS detector and segment the beats, fix the beat plotting function below (may not need this anymore, check get_sig2beats)
## 3. store the PPG (and ECG?) beats in npz (2D matrix) (DONE)
## 4. appned a column to df_merged that point to the matrix (file name) (DONE)
## 5. parallel computing (DONE)

In [41]:
# df_merged.shape

In [57]:
def process_row(row, index, inputdir, outputdir_recording):
    



    # 1. get header_dict
    rec_id = row['deBedTime']
    debug_message = '[perfecto]'

    print('\n==================  working on rec_id: {} =================='.format(rec_id) )
    inputdir_rec = inputdir + rec_id + '/'

    
    # print(os.listdir(inputdir_rec))
    


    # continue
    # header_dict = get_header_valid(inputdir_rec) # None or some dictionary data
    header_dict = get_header_valid_waveform(inputdir_rec, row) # None or some dictionary data


    # print(header_dict)

    # 2. continue if waveform is not within the header_dict timing range
    if header_dict is None:
        debug_message = '[DNE] wrong recording'
        if verbose:
            print(debug_message)
            print('\t', row['deBedTime'], row['spo2'], row['sao2'], row['times'], row['Black'], row['White'])
    
    else:
        if 'SPO2' not in list(header_dict['unit_dict'].keys()):
            debug_message = '[DNE] SPO2 channel not in waveform'
            if verbose:
                print(debug_message)

    if 'perfecto' in debug_message:

        # 3. get the waveform data
        df_bed, _ = get_df_bed(rec_id, inputdir, header_dict['header_id']) # RX limiting factor
        df_bed = df_bed[['I', 'SPO2', 'time']]

        

        
        
        # 4. get only a small piece of the waveform data
        sync_start = row['times'] - timedelta(minutes=waveform_window)
        sync_end = row['times'] + timedelta(minutes=waveform_window)

        i_sync_start = int((sync_start - header_dict['waveform_start']).total_seconds() * header_dict['Fs'])
        i_sync_end = int((sync_end - header_dict['waveform_start']).total_seconds() * header_dict['Fs'])

        df_bed = df_bed[i_sync_start:i_sync_end]


    
        # 5. check on the waveform data. Continue if the signals are off
        if df_bed.shape[0] < waveform_window*60*header_dict['Fs']/2:
            debug_message = '[Inadequate] data size smaller than {:.1f}s'.format( waveform_window/2 )
            if verbose:
                print(debug_message)
                print('\t', header_dict['header_name'], header_dict['waveform_start'],header_dict['waveform_end'],header_dict['t_dur(hr)'])
                print('\t', row['DateStart'], row['DateEnd'])
                print('\t', df_bed['time'].max()-df_bed['time'].min() )



    
    if 'perfecto' in debug_message:
        # get the waveform, segment it, store it     
        PPGmatrix, ECGmatrix, debug_message = df_bed2PPGmatrix(df_bed) # df_bed will be resampled to FS_RESAMPLE
        # print(PPGmatrix.shape, ECGmatrix.shape)
        # sys.exit()
        
        # select one of the window randomly
        PPGmatrix = PPGmatrix[:,np.random.randint(PPGmatrix.shape[1])]
        ECGmatrix = ECGmatrix[:,np.random.randint(ECGmatrix.shape[1])]
        # print(PPGmatrix.shape)

        if 'perfecto' not in debug_message:
            PPGmatrix_name = 'DNE'
            ECGmatrix_name = 'DNE'
        else:
            PPGmatrix_name = 'PPGmatrix_{}_{}_{}'.format(rec_id, header_dict['header_id'], index)  
            data_saver(PPGmatrix, PPGmatrix_name, outputdir_recording)
            
            ECGmatrix_name = 'ECGmatrix_{}_{}_{}'.format(rec_id, header_dict['header_id'], index)  
            data_saver(ECGmatrix, ECGmatrix_name, outputdir_recording)
            # data_saver(PPGmatrix, 'data', outputdir_recording)
        # PPGmatrix = data_loader('sig_segment_A035-0519626285_0000_7581', '../../data/stage1/waveform/')
        # row = pd.concat([row, pd.Series({'matrix_dir': matrix_dir, 'index': index, 'debug_message': debug_message})])

    # if 'perfecto' in debug_message:
        # row = pd.concat([row, pd.Series({'matrix_dir': matrix_dir, 'index': index, 'debug_message': debug_message})])
    else:
        PPGmatrix_name = 'DNE'
        ECGmatrix_name = 'DNE'
        
    row = pd.concat([row, pd.Series({'PPGmatrix_name': PPGmatrix_name, 'ECGmatrix_name': ECGmatrix_name, 'index': index, 'debug_message': debug_message})])

    row =  pd.DataFrame(row).T
    row.to_feather(outputdir_recording+'df_demographic_processed_{}.feather'.format(index))

    
    return row
    # df_demographic_processed = pd.concat([df_demographic_processed, pd.DataFrame(row).T])
    # df_demographic_processed.reset_index(drop=True).to_csv(outputdir_recording+'df_demographic_processed.csv.gz', index=False, compression='gzip')


In [43]:
df_merged

,deBedTime,DateStart,DateEnd,PAT_ID,BED_LOCATION_START,BED_LOCATION_END,White,Black,Race String,Time Difference,Time Found,True Time Found,CSN Exist Flag,CSN,BED Time Difference,spo2,sao2,times
0,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7998.0,94.5,92.4,2018-01-10 17:16:00
1,A140-0515603029,2018-01-10 16:50:29,2018-01-10 19:03:47,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1531.0,2018-01-10 17:16:00,2017-12-18 21:16:00,True,4.991014e+10,7998.0,96.0,92.4,2018-01-10 18:16:00
2,A140-0515620983,2018-01-10 21:49:43,2018-01-21 21:51:58,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1577.0,2018-01-10 22:16:00,2017-12-18 21:16:00,True,4.991014e+10,950535.0,95.0,93.9,2018-01-10 22:16:00
3,A140-0515620983,2018-01-10 21:49:43,2018-01-21 21:51:58,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1577.0,2018-01-10 22:16:00,2017-12-18 21:16:00,True,4.991014e+10,950535.0,94.0,93.9,2018-01-10 23:16:00
4,A140-0515620983,2018-01-10 21:49:43,2018-01-21 21:51:58,94166433.0,2018-01-10 16:50:12,2018-01-18 21:15:32,True,False,Caucasian or White,-1577.0,2018-01-10 22:16:00,2017-12-18 21:16:00,True,4.991014e+10,950535.0,93.5,93.9,2018-01-11 00:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149057,A063-0493131179,2017-04-25 14:39:39,2017-05-18 20:01:09,47565503.0,2017-04-12 17:38:13,2017-05-18 16:05:00,True,False,Caucasian or White,-1521.0,2017-04-25 15:05:00,2017-04-08 21:05:00,True,1.326399e+10,2006490.0,98.0,99.1,2017-05-16 01:05:00
149058,A063-0493131179,2017-04-25 14:39:39,2017-05-18 20:01:09,47565503.0,2017-04-12 17:38:13,2017-05-18 16:05:00,True,False,Caucasian or White,-1521.0,2017-04-25 15:05:00,2017-04-08 21:05:00,True,1.326399e+10,2006490.0,98.0,99.1,2017-05-16 02:05:00
149059,A063-0493131179,2017-04-25 14:39:39,2017-05-18 20:01:09,47565503.0,2017-04-12 17:38:13,2017-05-18 16:05:00,True,False,Caucasian or White,-1521.0,2017-04-25 15:05:00,2017-04-08 21:05:00,True,1.326399e+10,2006490.0,100.0,99.1,2017-05-16 03:05:00
149060,A063-0493131179,2017-04-25 14:39:39,2017-05-18 20:01:09,47565503.0,2017-04-12 17:38:13,2017-05-18 16:05:00,True,False,Caucasian or White,-1521.0,2017-04-25 15:05:00,2017-04-08 21:05:00,True,1.326399e+10,2006490.0,94.0,99.1,2017-05-16 04:05:00


In [44]:
# df_demographic_processed = pd.read_csv(outputdir_recording+'df_demographic_processed.csv.gz').sort_values('index').reset_index(drop=True)
# df_demographic_processed.shape

# loop for parsing the rows (don't delete)

In [45]:
# if ~from_scratch:
#     print('hi')

In [46]:
# ~from_scratch

In [47]:
files_in_outputdir_recording = os.listdir(outputdir_recording)

In [ ]:
waveform_window = 0.5 # minute,  preceeding and proceeding
verbose=False
from_scratch = False

if from_scratch:
    df_demographic_processed = pd.DataFrame()
else:
    # df_demographic_processed = pd.read_csv(outputdir_recording+'df_demographic_processed.csv.gz').sort_values('index').reset_index(drop=True)
    df_demographic_processed = pd.read_feather(outputdir_recording+'df_demographic_processed.feather', columns=None, use_threads=True)

    print(df_demographic_processed.shape)
# df_demographic_processed



for index, row in df_merged.iterrows():
    # start = time.time()
    
    if not from_scratch: # if not from scatch, check if index is in df_demographic_processed['index']
        if index in df_demographic_processed['index']:
            # print(index)
            continue
            
        if 'df_demographic_processed_{}.feather'.format(index) in files_in_outputdir_recording:
            continue
            
            if os.path.getsize(outputdir_recording+'df_demographic_processed_{}.feather'.format(index))==0:
                continue
    
    row_processed = process_row(row, index, inputdir, outputdir_recording)
    
    
    # df = pd.read_feather(outputdir_recording+'df_demographic_processed_{}.feather'.format(index), columns=None, use_threads=True)

    # sys.exit()
    
    df_demographic_processed = pd.concat([row_processed, df_demographic_processed])
    # df_demographic_processed.reset_index(drop=True).to_csv(outputdir_recording+'df_demographic_processed.csv.gz', index=False, compression='gzip')
    df_demographic_processed.reset_index(drop=True).to_feather(outputdir_recording+'df_demographic_processed.feather')

    
    
    
    
    
    # sys.exit()

    # continue

    # end = time.time()
    # print(end - start)

(1210, 22)

==================  working on rec_id: A060-0516834366 ==================

==================  working on rec_id: A060-0516834366 ==================

==================  working on rec_id: A060-0516834366 ==================

==================  working on rec_id: A060-0516834366 ==================

==================  working on rec_id: A060-0516834366 ==================

==================  working on rec_id: A060-0516834366 ==================

==================  working on rec_id: A060-0516834366 ==================

==================  working on rec_id: A060-0516834366 ==================

==================  working on rec_id: A060-0516834366 ==================

==================  working on rec_id: A132-0522270480 ==================

==================  working on rec_id: A132-0522270480 ==================

==================  working on rec_id: A012-0514952815 ==================

==================  working on rec_id: A012-0514952815 ==================

=============

In [ ]:
# ==================  working on rec_id: A088-0516752917 ==================
# row_processed

10522

In [ ]:
df_demographic_processed

In [ ]:
sys.exit()

# parallel computing

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

import time

start = time.time()

df_demographic_processed = pd.DataFrame()

num_cores = multiprocessing.cpu_count()

results = Parallel(n_jobs=num_cores)(delayed(process_row)(row, index, inputdir, outputdir_recording) for index, row in df_merged.iterrows())
df_demographic_processed = pd.concat(results)
df_demographic_processed.reset_index(drop=True).to_csv(outputdir_recording+'df_demographic_processed.csv.gz', index=False, compression='gzip')

end = time.time()
print(end - start)

In [ ]:
df = pd.read_csv(outputdir_recording+'df_demographic_processed.csv.gz')

In [ ]:
# results

In [ ]:
sys.exit()

# test on one random file

# get the header file

In [ ]:
rec_id = list(recording_lookup.keys())[selected_id]

bed_id = rec_id.split('-')[0]
subject_id = rec_id.split('-')[1]

header_id = recording_lookup[rec_id]
header_name = rec_id+'_{}.hea'.format(header_id)

# get recording directory
inputdir_rec = inputdir + rec_id + '/'

# get header file directory
headerdir = inputdir_rec + header_name
print(headerdir)

# load header file
with open(headerdir, 'r') as f:
    header = f.readlines()

header_dict = header_extract(header)

In [ ]:
header_dict['header_id'] = header_id

In [ ]:
header_dict

In [ ]:
# recording_lookup

# get raw waveform data

In [ ]:
df_bed, _ = get_df_bed(rec_id, inputdir, header_dict['header_id'])
df_bed

# duration of data

In [ ]:
t_arr = np.arange(df_bed.shape[0])/header_dict['Fs']

df_bed['time'] = t_arr
df_bed['time'] = df_bed['time']-df_bed['time'].values[0]

t_dur = df_bed.shape[0]/header_dict['Fs']
print('t_dur: {:.2f}s'.format(t_dur))

In [ ]:
header_dict['unit_dict']

In [ ]:
plot_all_sync(df_bed, rec_id, header_dict, t_start=300, t_end=400, plt_scale=0.3, fig_name=None, outputdir=None, show_plot=True)


In [ ]:
# extract beat matrix

In [ ]:
seg_duration = 30 * 60 # sec

Fs = header_dict['Fs']
N_segs = int((df_bed.shape[0]/Fs)//seg_duration)
verbose = True

df_features_bed = pd.DataFrame()

for i_seg in range(N_segs):

    if i_seg>3:
        continue

    if verbose:
        print('i_seg: ', i_seg)

    i_start = int(i_seg*seg_duration*Fs)
    i_end = int((i_seg+1)*seg_duration*Fs)

    df_seg = df_bed[i_start:i_end].copy()

    if df_seg.shape[0]==0:
        continue
    if np.mean(np.diff(df_seg['SPO2'])==0)>signal_quite_threshold:
        continue
    if np.mean(np.diff(df_seg['I'])==0)>signal_quite_threshold:
        continue
    if np.mean(np.diff(df_seg['II'])==0)>signal_quite_threshold:
        continue
    if np.mean(np.diff(df_seg['III'])==0)>signal_quite_threshold:
        continue
    if np.mean(np.diff(df_seg['V'])==0)>signal_quite_threshold:
        continue


    norm1 = np.abs(get_ecg_IP_norm(df_seg['I'].values, df_seg['II']))
    norm2 = np.abs(get_ecg_IP_norm(df_seg['I'].values, df_seg['III']))
    norm3 = np.abs(get_ecg_IP_norm(df_seg['I'].values, df_seg['V']))

    if (norm1 < 0.3) or (norm2 < 0.3) or (norm3 < 0.3):
        if verbose:
            print('\tbad ECG')
        continue


    sig_name = 'I'
    ECG = df_seg[sig_name].values


    # ECG = df[sig_name].values

    fig_name = 'ECG_diagnostics_'+sig_name
    # QRS_detector_dict = task_HR_detector(ECG, Fs, fig_name=fig_name, outputdir=None, show_plot=True)
    QRS_detector_dict = task_HR_detector(ECG, Fs, fig_name=fig_name, outputdir=None, show_plot=False)

#     t_ecg = np.arange(ECG.shape[0])/Fs
#     hr_interp = np.interp(t_ecg, QRS_detector_dict['ts_hr'], QRS_detector_dict['hr'])


    # segment the beats
    beats_dict = segment_df(df_seg, QRS_detector_dict, Fs)
    # mask_ppg, ol_rate = clean_PPG(beats_dict['SPO2'], beats_dict['SPO2'].mean(axis=1), Fs)
    # beats_dict = masking_beats_dict(beats_dict, mask_ppg)

    sys.exit()

In [ ]:
# TODO: check segment_df, validate the beat plotting function again

In [ ]:
get_filt_df

In [ ]:
# plt.plot(beats_dict['I'][:50, :], color='gray', alpha=0.5)
# plt.show()

In [ ]:
# beats_dict['I'].shape

In [ ]:
plt.plot(beats_dict['SPO2'][:50, :], color='gray', alpha=0.1)
plt.show()

In [ ]:
# # create a list
# prime_numbers = [2, 3, 5, 7, 9, 11]

# # remove 9 from the list
# prime_numbers.remove(1)

In [ ]:
# prime_numbers

In [ ]:
plot_all_sync(df_seg, rec_id, header_dict, t_start=300, t_end=400, plt_scale=0.3, fig_name=None, outputdir=None, show_plot=True)


# # TODO: need to renovate this function so it can work again


In [ ]:
def plot_ALL_beats2(beats_dict, beats_id, subject_id, Fs, show_good=None, fig_name=None, outputdir=None, show_plot=False):
    beats_id = [1,2,3,4,5,6, 7]
    beats_names = ['I', 'II', 'III', 'V', 'SPO2', 'PPG_DC', 'AR2']
    unit_dict = header_dict['unit_dict']
    unit_dict['PPG_DC'] = ''
    t_beat = np.arange(beats_dict['I'].shape[0])/Fs

    fig = plt.figure(figsize=(16, 10), dpi=80)
    fontsize = 20
    alpha = 0.03


    # for (beat_name, beat_i) in zip(beats_dict, beats_id):
    for (beat_name, beat_i) in zip(beats_names, beats_id):

        beats = beats_dict[beat_name]

        ax = fig.add_subplot(3, 4, beat_i)
        ax.set_title(beat_name+'\n', fontsize=fontsize)

        show_good = True
        if show_good is not None:


            template = beats_dict[beat_name].mean(axis=1)
            if beat_name=='SPO2':
                mask_all, ol_rate = clean_PPG(beats, template, Fs)
                ax.set_title(beat_name+'\nacception_rate:{:.2f}'.format(1-ol_rate), fontsize=fontsize)


        ax.plot(t_beat, beats, color='gray', alpha=alpha)
        ax.plot(t_beat, np.mean(beats,axis=1), color='firebrick', linewidth=3)

        if 'SPO2' in beat_name or 'I' == beat_name or 'II' == beat_name or 'III' == beat_name or 'V' == beat_name:
            beats_mean = np.mean(beats,axis=1)

            ymin = beats_mean.mean() - beats_mean.std()*5
            ymax = beats_mean.mean() + beats_mean.std()*5
            ax.set_ylim(ymin, ymax)

        ax.tick_params(axis='both', which='major', labelsize=13)
        ax.set_ylabel(unit_dict[beat_name], fontsize=fontsize-3)
        ax.set_xlabel('time (sec)', fontsize=fontsize)



    fig.tight_layout()

    # if outputdir is not None:

    #     if fig_name is None:
    #         fig_name = 'beats_ensemble_sub{}'.format(subject_id)

    #     if not os.path.exists(outputdir):
    #         os.makedirs(outputdir)
    #     fig.savefig(outputdir + fig_name+'.png', transparent=False)

    # if show_plot == False:
    #     plt.close(fig)
    #     pyplot.close(fig)
    #     plt.close('all')